 10am, 1pm, 4pm, 6pm, 10pm

In [1]:
import pandas as pd
import subprocess
import concurrent.futures
import re

In [2]:
servers = {
            'Michigan Technological University Michigan': 'www.mtu.edu',
            'U. of Wisconsin at Madison Wisconsin': 'www.wisc.edu',
            'U. of Minnesota Minnesota': 'www.umn.edu',
            'Iowa State U. Iowa': 'www.iastate.edu',
            'U. of Nebraska at Lincoln Nebraska': 'www.unl.edu',
            'Colorado University Colorado': 'www.cu.edu',
            'U. of Utah Utah': 'www.utah.edu',
            'U. of Cambridge UK': 'www.cam.ac.uk',
            'Ecole Polytechnique France': 'www.polytech-lille.fr',
            'Temple University at Japan Japan': 'www.tuj.ac.jp',
            'The University of Melbourne Australia': 'www.unimelb.edu.au',
            'American University of Rome Rome': 'www.aur.edu',
            'Indian Institute of Science India': 'www.iisc.ac.in'
           }

urls = list(servers.values())

In [3]:
def execute_command(command):
    completed_process = subprocess.run(command, shell=True, capture_output=True, text=True)
    return completed_process.stdout, completed_process.stderr

output = {}

def process_server(server):
    try:
        ping_cmd = f"ping -c 100 {server}" 
        ping_stdout, ping_stderr = execute_command(ping_cmd)
        print ('Completed pinging: ', server)

        traceroute_cmd = f"traceroute {server}"
        traceroute_stdout, traceroute_stderr = execute_command(traceroute_cmd)
        print ('Completed traceroute: ', server)

        return {
            "ping_stdout": ping_stdout,
            "ping_stderr": ping_stderr,
            "traceroute_stdout": traceroute_stdout,
            "traceroute_stderr": traceroute_stderr
            }

    except Exception as e:
        print(f"Error processing {server}: {e}")
        return None

    

with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = {executor.submit(process_server, server): server for server in urls}
    for future in concurrent.futures.as_completed(futures):
        server = futures[future]
        try:
            output[server] = future.result()
        except Exception as e:
            print(f"Error processing {server}: {e}")


Completed pinging:  www.iastate.edu
Completed pinging:  www.mtu.edu
Completed pinging: Completed pinging:  www.cu.edu www.wisc.edu
Completed pinging:  www.aur.edu
Completed pinging: 
 Completed pinging:  www.umn.edu
www.unl.edu
Completed pinging: Completed pinging:  www.cam.ac.uk
Completed pinging:   www.tuj.ac.jpwww.utah.edu

Completed pinging:  www.unimelb.edu.au
Completed pinging:  www.polytech-lille.fr
Completed traceroute:  www.unimelb.edu.au
Completed traceroute:  www.mtu.edu
Completed traceroute:  www.utah.edu
Completed traceroute:  www.polytech-lille.fr
Completed pinging:  www.iisc.ac.in
Completed traceroute:  www.wisc.edu
Completed traceroute:  www.cam.ac.uk
Completed traceroute:  www.tuj.ac.jp
Completed traceroute:  www.iisc.ac.in
Completed traceroute:  www.umn.edu
Completed traceroute:  www.unl.edu
Completed traceroute:  www.iastate.edu
Completed traceroute:  www.aur.edu
Completed traceroute:  www.cu.edu


In [4]:
data = {'server'  : [],
        'rtt_min' : [],
        'rtt_max' : [],
        'rtt_avg' : [],
        'avg_pkt_loss': [],
        'num_hops' : [],
        'distinct_domains' : []
        }

for url, out in output.items():
    data['server'].append(url)
    res = out['ping_stdout']
    rtt_min, rtt_avg, rtt_max = res.split('min/avg/max/stddev = ')[-1].split('/')[:3]
    data['rtt_min'].append(rtt_min)
    data['rtt_avg'].append(rtt_avg)
    data['rtt_max'].append(rtt_max)
    data['avg_pkt_loss'].append(res.split('%')[0].split(' ')[-1])
    res = out['traceroute_stdout']
    data['num_hops'].append(len(res.split('\n')))

    ips = (re.findall(r'\((.*?)\)', out['traceroute_stdout']))
    ips = ['.'.join(ip.split('.')[0:2]) for ip in ips]
    data['distinct_domains'].append(len(set(ips)))

In [9]:
df_10am = pd.DataFrame(data)
df_10am.set_index('server', inplace=True)

In [10]:
df_10am

,rtt_min,rtt_max,rtt_avg,avg_pkt_loss,num_hops,distinct_domains
server,,,,,,
www.unimelb.edu.au,12.595,158.404,30.696,0.0,6,5
www.mtu.edu,13.339,103.998,30.237,0.0,12,9
www.utah.edu,52.795,221.702,82.822,0.0,46,12
www.polytech-lille.fr,102.332,197.975,125.355,0.0,15,9
www.wisc.edu,12.144,103.597,29.693,0.0,27,9
www.cam.ac.uk,99.159,202.601,123.400,0.0,65,9
www.tuj.ac.jp,25.843,123.761,48.800,0.0,70,8
www.iisc.ac.in,226.123,532.502,295.573,0.0,76,5
www.umn.edu,19.745,108.825,38.591,0.0,68,6
